In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
)

template = PromptTemplate.from_template("What is the capital of {country}?")

prompt = template.format(country="France")

chat.predict(prompt)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "Hello, my name is {name}"),
    ("human", "what is the capital of {country}? And what is your name?")
])

prompt = template.format_messages(language="korean", name="Jamtol", country="Korea")

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, my name, is Jamtol")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

In [ ]:
chain = template | chat | CommaOutputParser()
chain.invoke({"question": "what are the pokemons?", "max_items": 5})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a chef. You create a recipe for a dish."),
    ("human", "{dish}")
])

chef_chain = chef_template | chat

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef. You find alternative ingredients for a dish."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"dish": "Spaghetti Carbonara"})

### 4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

In [ ]:
example = [
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "What do you know about France?",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    }
]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    suffix="Question: What do you know about {Country}?",
    input_variables=["Country"],
)

chain = prompt | chat

chain.invoke({"Country": "Germany"})

### 4.2 FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

example = [
    {
        "question": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "France",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    }
]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    examples=example,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about countries."),
    example_prompt,
    ("human", "What do you know about {question}?"),
])

chain = final_prompt | chat

chain.invoke({"question": "Thailand"})

### 4.3 LengthBasedExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "What do you know about France?",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    },
    {
        "question": "What do you know about Korea?",
        "answer": """
I know this:
Capital: Seoul
Language: Portuguese
Currency: Real
Food: Feijoada
"""
    }
]

class RandomExampleSelector(BaseExampleSelector):
    """Select random examples to use based on the input variables."""
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Question: What do you know about {Country}?",
    input_variables=["Country"],
)

prompt.format(Country="Brazil")

### 4.4 Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)


intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is a example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts = prompts,
)

chain = full_prompt | chat

chain.invoke({
    "character": "a pirate", 
    "example_question": "What is your location?", 
    "example_answer": "Arrrr! That is a secret! Arrrr!", 
    "question": "What is your favorite food?"
})

### 4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.invoke("How do you make italian pasta")

In [ ]:
chat.invoke("How do you make italian pasta")

### 4.6 Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)
    print(usage.total_tokens)
    print(usage.prompt_tokens)
    print(usage.completion_tokens)
    print(usage.total_cost)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=456,
#     model = "gpt-3.5-turbo",
# )

# chat.save("model.json")

chat = load_llm("model.json")

chat

### 5.0 ConversationBufferMemory

모든 대화 내용을 저장하는 방식 [대화가 길어질 수록 보내야 정보량이 늘어 비효율적]

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=False) # True로 설정하면 AI 모델이 활용 가능

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

### 5.1 ConversationBufferWindowMemory

설정한 개수만큼만의 최근 대화 기억

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)
add_message(6,6)


memory.load_memory_variables({})

### 5.2 ConversationSummaryMemory

초반에는 llm을 사용하다보니 토큰을 좀 사용하겠지만, Context가 길어지면 길어질수록 효율적임

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jamtol", "Nice to meet you!")

In [ ]:
add_message("South Korea is pretty", "I wish I could go!!")

In [ ]:
get_history()

### 5.3 ConversationSummaryBufferMemory

최근 대화는 자체 보존하고 기준 초과하는 옛 대화는 요약해 기록

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi, my name is Jamtol", "Hello Jamtol, it's nice to meet you!")
add_message("I'm doing well! Just having a conversation with an AI", "That's great to hear Jamtol, how can I help you today?")

get_history()


In [ ]:
add_message("korea is amazing", "I wish I could go!!")
get_history()

### 5.4 ConversationKGMemory

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi, my name is Jamtol, I'm from Korea", "Hello Jamtol, it's nice to meet you!")


In [ ]:
memory.load_memory_variables({"input": "who is Jamtol"})

In [ ]:
add_message("Jamtol likes ham", "Wow that is great!")

In [ ]:
memory.load_memory_variables({"input": "what does Jamtol like?"})

### 5.5 Memory on LLMChain

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    memory_key = "chat_history"
)

template = """
You are a helpful AI assistant.

{chat_history}

human: {question}
you:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Jamtol")
chain.predict(question="I'm from Korea")
chain.predict(question="I'm a student")
chain.predict(question="I'm a student")

In [ ]:
memory.load_memory_variables({})

### 5.6 Chat Based Memory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

### 5.7 LCEL Based Memory

In [19]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [20]:
invoke_chain("My name is Jamtol")

content='Hello Jamtol! How can I assist you today?'


In [21]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='My name is Jamtol'),
  AIMessage(content='Hello Jamtol! How can I assist you today?')]}

In [22]:
invoke_chain("What is my name?")

content='Your name is Jamtol. How can I assist you today, Jamtol?'


### 6.0 Introduction

#### RAG
질문을 했을때 2가지 작업을 진행함
프롬프트를 제공 + 문서에 있는 질문 참고 내용 제공



### 6.1 Data Loaders and Splitters

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("files/chapter_one.txt")
loader.load()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)

loader = UnstructuredFileLoader("files/chapter_one.txt")

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(len(splitted_docs))
print(splitted_docs)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("files/chapter_one.txt")

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(len(splitted_docs))
print(splitted_docs)

### 6.2 Tiktoken

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=10,
)

loader = UnstructuredFileLoader("files/chapter_one.txt")

splitted_docs = loader.load_and_split(text_splitter=splitter)

print(len(splitted_docs))



### 6.3 Vectors

AI에 문서를 있는 그대로 넣지 않는다  
오히려 벡터 차원에서 각 단어들 간의 유사도를 계산하여 원하는 데이터와 유사도가 높은 데이터들을 LLM에 넣는 방식을 쓴다  
이것이 RAG

### 6.4 Vector Store

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents(["Hi", "Hello", "how are you?"])

print(len(vector[0]))

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [18]:
results = vectorstore.similarity_search("where does winston live")

results

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liqu

### 6.5 Langsmith

### 6.6 RetrievalQA

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "refine",
    retriever = vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'The additional context provided sheds light on Winston\'s living situation within Victory Mansions, specifically his attempt to find a private space away from the constant surveillance of the telescreen. This detail emphasizes the oppressive nature of the Party\'s control in Winston\'s environment. The description of the book he acquires also highlights the scarcity of certain items and the illicit nature of obtaining such possessions in a society where individuality and free thought are suppressed. The inclusion of these details further illuminates the bleak and restrictive atmosphere of Victory Mansions and the larger dystopian world depicted in "1984."'

### 6.7 Recap

### 6.8 Stuff LCEL Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}"),
])

chain = {"context":retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is a run-down apartment complex with glass doors, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The building has a faulty lift, and Winston\'s flat is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with the caption "BIG BROTHER IS WATCHING YOU" hangs on the wall.')

### 6.9 Map Reduce LCEL Chain

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

AIMessage(content='Two ministries are mentioned in the provided text.')

### 6.10 Recap